# Basic setup for fetching data from the facebook api

### Remember to add an active access token below and adjust the parameters and fields if necessary

In [ ]:
# must be specified to be able to use methods of package
import sys
sys.path.append('../src') 
# import packages
import numpy as np
import pandas as pd
from political_ads.api_request import get_json_response
from political_ads.helper import *

# insert your access token here (expires every 2 hours)
access_token = "EAAD3So8oorMBAJVZCxzID8UzfsJ5N4lSBtvXyQy3HJv7mgdb8QfL223XlF1gwCeY628gyKFuZC2ttI5SrYZA41eLuPGqzIWcsXyOXrhTF8nkrZBLUP6PqwlAp2Glqe4QYhObaUB8X5BBjCx8Lagu64wFz5yCcH6oJHlosEgCW2k69EZBadiMG6dmTJ9uVxZB5zYss0U0ulzGVlBQealWYcO6XIeXsdENCEmxL8BkfXw99QieOPcz2WydTHZAdvvZBAMZD"

# PARAMETERS NECESSARY
ad_type = "POLITICAL_AND_ISSUE_ADS"
ad_reached_countries = ['US'] # Facebook delivered the ads in these countries. Provided as ISO country codes.
search_terms = "climate" # The terms to search for in your query. We treat a blank space as a logical AND and search for both terms and no other operators. The limit of your string is 100 characters or less.
limit = 500

# FIELDS to specify your results
fields = ["ad_creation_time", "ad_creative_body","spend", "impressions", "delivery_by_region", "demographic_distribution","page_id", "page_name","bylines","id"]

# see here for more parameters and fields: https://www.facebook.com/ads/library/api/?source=archive-landing-page 

input_url = f"https://graph.facebook.com/v12.0/ads_archive?search_terms={search_terms}&ad_type={ad_type}&ad_reached_countries={ad_reached_countries}&fields={fields}&limit={limit}"

response = get_json_response(input_url, access_token)

data = pd.DataFrame(response["data"])


In [ ]:
data

In [ ]:
'''
Data Generation
Loop to continuously fetch data, until there exists a next page
'''
import time

counter = 0
while "next" in response["paging"]:
    if counter == 50:
        break
    response = get_json_response(response["paging"]["next"], access_token)
    new_data = pd.DataFrame(response["data"])
    data = data.append(new_data, ignore_index=True)
    counter += 1
    time.sleep(10)
    print(f"Iteration number {counter}")

# Could go up to 70.000 entries
# Reached iteration 140

In [ ]:
# Save data as csv
data.to_csv("..\\data\\exported_df.csv", index=False, header=True)

In [ ]:
# Read csv
data = pd.read_csv("..\\data\\exported_df.csv")

In [ ]:
print("count before removing duplicates: " + str(data.count()["id"]))

data.drop_duplicates(subset=["ad_creative_body"], inplace=True)

print("count after removing duplicates: " + str(data.count()["id"]))


In [ ]:

def string_to_dict(dict_string):
    import json 
    # Convert to proper json format
    dict_string = dict_string.replace("'", '"')
    return json.loads(dict_string)

diction = string_to_dict(data["impressions"][0])
diction["lower_bound"]

In [ ]:
# Apply transformations to dataset
data["spend"] = data["spend"].apply(lambda x: string_to_dict(x))
data["impressions"] = data["impressions"].apply(lambda x: string_to_dict(x))

In [ ]:
# def transform_range(entry: dict):
#     return int(int(entry["lower_bound"]) + int(entry["upper_bound"])) / 2

In [ ]:
# Apply transformations to dataset
data["spend"] = data["spend"].apply(lambda x: transform_range(x))
data["impressions"] = data["impressions"].apply(lambda x: transform_range(x))

In [ ]:
data["demographic_distribution"][0]

In [ ]:
# Convert date to datetime timestamp object
data["ad_creation_time"] = pd.to_datetime(data["ad_creation_time"])
data["ad_creation_time"]

In [ ]:
'''
Visualization attempts
https://pandas.pydata.org/pandas-docs/dev/getting_started/intro_tutorials/09_timeseries.html
'''
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting spending over time and impressions over time 
fig, axes = plt.subplots(2,1,figsize=(8,4))
data.plot(x="ad_creation_time", y="impressions", ax=axes[0])
data.plot(x="ad_creation_time", y="spend", ax=axes[1])

In [ ]:
'''
Amount spend by facebook page
'''
by_page = data.groupby("page_name").agg(
    # Aggregate no of ads
    no_ads = ('id', 'count'),
    # Aggregate sum of spend
    sum_spend = ('spend', 'sum'),
    # Average number of impressions per ad
    avg_impressions = ('impressions', 'mean')

).reset_index()

by_page[by_page["page_name"] == "Sierra Club"]


In [ ]:
data[data["page_name"] == "Sierra Club"]

In [ ]:
'''
Data Generation
Loop to continuously fetch data, until there exists a next page
'''
import time

counter = 0
while "next" in response["paging"]:
    response = get_json_response(response["paging"]["next"], access_token)
    new_data = pd.DataFrame(response["data"])
    data = data.append(new_data, ignore_index=True)
    counter += 1
    time.sleep(10)
    print(f"Iteration number {counter}")

# Could go up to 70.000 entries
# Reached iteration 140

In [ ]:
data.to_csv("..\\data\\exported_df.csv", index=False, header=True)

In [ ]:
'''
Read data sample
'''
sample = pd.read_csv("..\\data\\exported_df.csv")

sample

In [10]:
# Scraper experiement
# https://github.com/kevinzg/facebook-scraper

from facebook_scraper import get_page_info

page = get_page_info(account="tomcarper")

page


{'name': 'Senator Tom Carper',
 'identifier': 100044593172202,
 'url': 'https://www.facebook.com/tomcarper',
 'image': 'https://scontent.ffra2-1.fna.fbcdn.net/v/t1.6435-1/fr/cp0/e15/q65/243163270_413515513478218_1995978801116196960_n.jpg?_nc_cat=109&ccb=1-5&_nc_sid=a12de3&_nc_ohc=b_fBITUPV_MAX99J8MD&_nc_ht=scontent.ffra2-1.fna&oh=21f6aa609db9f54d3171eb9eebe6a6d6&oe=6198ED3F',
 'sameAs': 'http://carper.senate.gov http://youtube.com/SenatorCarper http://flickr.com/SenatorTomCarper http://twitter.com/#!/SenatorCarper',
 'type': 'Person',
 'likes': 17074}